In [1]:
import sys  # For running system level commands
import yaml  # For reading the config file
import os  # For path tools
import requests  # For symphony client
import symphony_client  # For connecting to Symphony region
import purestorage  # For running Pure Storage commands
from pip._vendor.distlib.compat import raw_input
import datetime
from pytz import timezone
import pytz  # To add timezone to datetime

In [2]:
!{sys.executable} -m pip install purestorage
!{sys.executable} -m pip install pyyaml

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
print(u"[INIT] Initialising script.")
# Configure environment
tz_utc = pytz.timezone("UTC") # Set timezone for data source
current_day= ["ISO Week days start from 1","Mon","Tues","Wed","Thurs","Fri","Sat","Sun"]
rootpath = os.path.dirname(os.path.realpath('__file__'))  # Get the root path

[INIT] Initialising script.


In [4]:
# Import config file data
with open(rootpath + '/config.yml', 'r') as stream:
    try:
        config = yaml.safe_load(stream)
        print(u" [\u2713] Config file loaded.")
    except yaml.YAMLError as exc:
        print(u" [\u2717] Could not load the config file.")
        print(exc)
        exit()

 [✓] Config file loaded.


In [5]:
# Configure Stratoscale API connection
symp_url = "https://" + config["region_access"]["ipaddress"]
symp_domain = config["region_access"]["cloud_domain"]
symp_user = config["region_access"]["cloud_user"]
symp_password = config["region_access"]["cloud_password"]
symp_project = config["region_access"]["project"],

my_session = requests.Session()
my_session.verify = False

try:
    client = symphony_client.Client(url=symp_url, session=my_session)
    client_login = client.login(domain=symp_domain, username=symp_user, password=symp_password,project=symp_project)
    print(u" [\u2713] Stratoscale user region [{}] session established.".format(symp_url))
except:
    print(u" [\u2717] Could not connect to the Stratosacle region [{}] as user".format(symp_url))
    exit()


 [✓] Stratoscale user region [https://192.168.101.100] session established.


In [6]:
#Configure Pure Storage API Connection
pureip = str(config['purestoragearray']['ipaddress'])
puretoken = str(config['purestoragearray']['apitoken'])
array = purestorage.FlashArray(pureip, api_token=puretoken)
try:
    array_info = array.get()
    print(u" [\u2713] FlashArray {} [{}] (version {}) REST session established!".format(array_info['array_name'],
                                                                                pureip, array_info['version']))
except:
    print(u" [\u2717] Could not connect to the Pure Storage array - IP [" + pureip + "]")
    exit()

 [✓] FlashArray dsclabpure02 [172.16.100.10] (version 5.1.6) REST session established!


In [7]:
# Get hostname list
the_hosts = []
host_dict = {
    u'stratoshortname' : "empty",
    u'stratolongname' : "empty",
    u'pure' : "empty",
    u'vols' : []
}
array_hosts = array.list_hosts()
strato_hosts = client.nodes.list()
for strato_host in strato_hosts:
    for array_host in array_hosts:
        if strato_host.name in array_host.get("name"):
            print " > found [{}]".format(strato_host.name)
            host_dict[u'stratoshortname'] = strato_host.name
            host_dict[u'stratolongname'] = strato_host.hostname
            host_dict[u'pure'] = array_host.get("name")
            hostvolumelist = array.list_host_connections(array_host.get("name"))
            host_dict[u'vols'] = []
            for hostvolume in hostvolumelist:
                host_dict[u'vols'].append(hostvolume.get("vol"))
            the_hosts.append(dict(host_dict))

 > found [dscr1p1c1nb]
 > found [dscr1p1c1nd]
 > found [dscr1p1c1na]
 > found [dscr1p1c1nc]


In [8]:
print "Get which volumes are currently on which hosts."
the_vols = []
vol_dict = {
    u'vol_id' : "empty",
    u'stratolongname' : "empty",
}

strato_volumes = client.meletvolumes.list()
for strato_volume in strato_volumes:
    try:
        strato_volume_host = strato_volume.attachedHost[0]
    except:
        strato_volume_host = "None"
    vol_dict[u'vol_id'] = "volume-{}-cinder".format(strato_volume.id)
    vol_dict[u'stratolongname'] = strato_volume_host
    the_vols.append(dict(vol_dict))

Get which volumes are currently on which hosts.


In [9]:
print "Compare Stratoscale volume connections to pure volume connections, and remove incorrect connections."
for vol in the_vols:
    print " {} [{}]".format(vol.get("vol_id"),vol.get("stratolongname"))
    for host in the_hosts:
        if host.get("stratolongname") == vol.get("stratolongname"):
            print "\tVolume should exist on this host [{}]".format(host.get("stratolongname"))
        else:
            if vol.get("vol_id") in host.get("vols"):
                print "\tDisconnecting volume from {}".format(host.get("stratolongname"))
                print "\t\t> {}\n\t\t> {}".format(host.get("pure"),vol.get("vol_id"))
                array.disconnect_host(host.get("pure"),vol.get("vol_id"))

Compare Stratoscale volume connections to pure volume connections, and remove incorrect connections.
 volume-9addd312-59e5-4002-a39e-1f3ea809f6f1-cinder [None]
 volume-6f6e67b0-02ac-4806-9da8-6d8d65b231fd-cinder [None]
 volume-9fc5979c-2204-4807-8b1c-b47bab7ed773-cinder [dscr1p1c1nb.node.strato]
	Volume should exist on this host [dscr1p1c1nb.node.strato]
 volume-bbd6cda7-b654-4009-a1a9-304e83f13887-cinder [dscr1p1c1nd.node.strato]
	Volume should exist on this host [dscr1p1c1nd.node.strato]
 volume-bbddddd1-b1e4-400b-b0c3-d2aa2e478638-cinder [None]
 volume-cfcd1e6a-1139-480e-a78b-e0f35b8d7125-cinder [None]
 volume-8147987a-cc6c-480f-a792-d6bc73a5e82e-cinder [dscr1p1c1na.node.strato]
	Volume should exist on this host [dscr1p1c1na.node.strato]
 volume-51f30eca-76ad-4818-89f1-d05ff60eda73-cinder [None]
 volume-d6e8db8b-0dbb-481f-85cf-7e5feabea193-cinder [None]
 volume-58c022e6-4447-4020-84b6-2dc443a42f00-cinder [dscr1p1c1nd.node.strato]
	Volume should exist on this host [dscr1p1c1nd.node.s

 volume-5b038e57-df5c-44db-95d8-b960473396ac-cinder [dscr1p1c1nc.node.strato]
	Volume should exist on this host [dscr1p1c1nc.node.strato]
 volume-23f514a4-578b-4cdf-8ce8-ca90dfeda052-cinder [dscr1p1c1na.node.strato]
	Volume should exist on this host [dscr1p1c1na.node.strato]
 volume-901a78e6-f2e7-4ce0-8369-9d336d412c46-cinder [None]
 volume-7f4b49c4-6773-4927-b822-736e21487c3a-cinder [dscr1p1c1nc.node.strato]
	Volume should exist on this host [dscr1p1c1nc.node.strato]
 volume-980ef59c-1820-44e3-ad9a-050d40a0812b-cinder [dscr1p1c1nd.node.strato]
	Volume should exist on this host [dscr1p1c1nd.node.strato]
 volume-d759c6eb-2cc6-44ea-8974-632d0d83e0af-cinder [None]
 volume-e49d32bc-3ab5-44ee-a1b4-28d0840fd6a6-cinder [None]
 volume-64b83a9c-8c9b-44f1-97f5-e277e18bb5aa-cinder [dscr1p1c1na.node.strato]
	Volume should exist on this host [dscr1p1c1na.node.strato]
 volume-679f2b0f-5a4f-44f4-b5c5-09e598e61e90-cinder [dscr1p1c1nc.node.strato]
	Volume should exist on this host [dscr1p1c1nc.node.str

In [10]:
# Disconnect sessions
array.invalidate_cookie()